In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [4]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [5]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [6]:
amazon_df

,title,price,rating,reviews,availability
0,Sony Playstation PS4 1TB Black Console,,4.7 out of 5 stars,"1,313 ratings",In Stock
1,"Playstation Sony 4, 500GB Slim System [CUH-221...",,4.6 out of 5 stars,334 ratings,Only 7 left in stock - order soon.
2,Newest Sony Playstation 4 Slim 1TB SSD Console...,,4.5 out of 5 stars,301 ratings,Only 6 left in stock - order soon.
3,Uplay Newest Play-Station 4 Console PS.4 with ...,,4.8 out of 5 stars,9 ratings,Only 13 left in stock - order soon.
4,Flagship Newest Play Station 4 1TB HDD Only on...,,4.6 out of 5 stars,193 ratings,Only 2 left in stock - order soon.
5,Sony PlayStation 4 (Latest Model)- 500 GB Jet ...,,3.6 out of 5 stars,7 ratings,Only 8 left in stock - order soon.
6,PlayStation 4 Slim 500GB Console [Discontinued],,4.5 out of 5 stars,"1,886 ratings",In Stock
7,RPlay Play-Station 4 PS4 1TB Slim Edition Jet ...,,3.7 out of 5 stars,28 ratings,Only 14 left in stock - order soon.
8,PlayStation 4 Slim 1TB Console - Black (Renewed),,4.1 out of 5 stars,"1,462 ratings",Only 1 left in stock - order soon.
9,PlayStation 4 500GB Console (Renewed),,4.1 out of 5 stars,926 ratings,In Stock
